In [1]:
import matplotlib.pyplot as plt
# ^^^ pyforest auto-imports - don't write above this line
# to remove errors 
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

2022-06-21 07:47:44.114381: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-21 07:47:44.114495: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.metrics import confusion_matrix, classification_report

Perceptrons - Core of ANN
   - A layer of Linear Threshold Units that are a type of artifical neuron
   - Given an input vector of features we train for optimal weights on each neuron s.t. our loss function is minimized
   - The underlying optimization problem is solved via Stochastic Gradient Descent
   

Using Iris dataset note the Attribute Information:
   1. sepal length in cm
   2. sepal width in cm
   3. petal length in cm
   4. petal width in cm
   5. class: Iris Setosa/Iris Versicolour/Iris Virginica

In [3]:
iris=pd.read_csv('/home/jbohn/jupyter/datasets/ML_Data/Iris/iris.csv',names=['sepal_length','sepal_width','petal_length','petal_width','target'])
features=['sepal_length','sepal_width','petal_length','petal_width']
iris

,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [4]:
X_train,X_test,Y_train,Y_test=train_test_split(iris[features],iris['target'],train_size=0.75)
X_train

,sepal_length,sepal_width,petal_length,petal_width
92,5.8,2.6,4.0,1.2
137,6.4,3.1,5.5,1.8
54,6.5,2.8,4.6,1.5
97,6.2,2.9,4.3,1.3
140,6.7,3.1,5.6,2.4
...,...,...,...,...
108,6.7,2.5,5.8,1.8
63,6.1,2.9,4.7,1.4
60,5.0,2.0,3.5,1.0
52,6.9,3.1,4.9,1.5


Now we are ready to fit a perceptron model on the training data. To find the parameterized perceptron model with the minimized loss we apply stochastic gradient descent.

This assumes our target function is differentiable and convex

Moreover our target function will separate our target space into m-subspaces with m-1 hyperplanes. By perceptron convergence thrm this will only converge if the data is linearly separable 


In [5]:
clf = Perceptron(tol=1e-3)
clf.fit(X_train.values, Y_train.values)
param_dict=clf.get_params()
param_dict
for key,value in param_dict.items():
    print(key, value)

alpha 0.0001
class_weight None
early_stopping False
eta0 1.0
fit_intercept True
l1_ratio 0.15
max_iter 1000
n_iter_no_change 5
n_jobs None
penalty None
random_state 0
shuffle True
tol 0.001
validation_fraction 0.1
verbose 0
warm_start False


In [6]:
score=clf.score(X_test, Y_test)
print("Accuracy on Test Data is", score)

Accuracy on Test Data is 0.9736842105263158


/home/jbohn/miniconda3/envs/research/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but Perceptron was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [7]:
Y_pred=clf.predict(X_test)
confusion_matrix(Y_test,Y_pred)

/home/jbohn/miniconda3/envs/research/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but Perceptron was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


array([[13,  0,  0],
       [ 0, 10,  1],
       [ 0,  0, 14]])

In [8]:
print(classification_report(Y_test,Y_pred))

                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        13
Iris-versicolor       1.00      0.91      0.95        11
 Iris-virginica       0.93      1.00      0.97        14

       accuracy                           0.97        38
      macro avg       0.98      0.97      0.97        38
   weighted avg       0.98      0.97      0.97        38



Multi-Layer Perceptrons 
   - Multiple layers of Perceptrons with either the same/different activation functions
   - We can think of this as finding the most likely transformation function (projection) of our feature space into our target space. 
   - I.e. our feature might not share the same space as our target but a nonlinear function of our feature might


In [18]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

Standardizing data to a normal distribution

In [138]:
scaler = StandardScaler()
scaler=scaler.fit(X_train)
scaled_X_train=scaler.transform(X_train)
scaled_X_test=scaler.transform(X_test)

Let's fit a MLP of 1 hidden layer where activation function is RELU and we numerically solve for the weight values using LBFGS Optimiazaiton and a L2 Regularization of $\alpha$. 

The loss function here is Log-Loss (Penalized)

In [188]:
clf = MLPClassifier(solver='lbfgs', alpha=0.0001,hidden_layer_sizes=(1))
model=clf.fit(scaled_X_train, Y_train)

In [189]:
model.get_params()


{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': 1,
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 200,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

In [190]:
print("Model Achieved Loss in",model.n_iter_, "iterations")

Model Achieved Loss in 111 iterations


In [191]:
model.score(scaled_X_test,Y_test)

1.0

In [193]:
Y_pred=clf.predict(scaled_X_test)
confusion_matrix(Y_test,Y_pred)

array([[13,  0,  0],
       [ 0, 11,  0],
       [ 0,  0, 14]])

Now fitted with logstic activation function

In [194]:
clf = MLPClassifier(solver='lbfgs',activation='logistic', alpha=0.0001,hidden_layer_sizes=(1))
model=clf.fit(scaled_X_train, Y_train)

In [195]:
print("Model Achieved Loss in",model.n_iter_, "iterations")

Model Achieved Loss in 60 iterations


In [196]:
model.score(scaled_X_test,Y_test)

1.0

In [197]:
Y_pred=clf.predict(scaled_X_test)
confusion_matrix(Y_test,Y_pred)

array([[13,  0,  0],
       [ 0, 11,  0],
       [ 0,  0, 14]])

Now let's include a different optimization function

In [198]:
clf = MLPClassifier(solver='adam',activation='logistic', alpha=0.001,hidden_layer_sizes=(1),max_iter=10000)
model=clf.fit(scaled_X_train, Y_train)

In [199]:
print("Model Achieved Loss in",model.n_iter_, "iterations")

Model Achieved Loss in 1996 iterations


In [200]:
model.score(scaled_X_test,Y_test)

0.631578947368421

In [201]:
Y_pred=clf.predict(scaled_X_test)
confusion_matrix(Y_test,Y_pred)

array([[13,  0,  0],
       [ 0, 11,  0],
       [ 0, 14,  0]])